In [1]:
import os
os.chdir('../') 

In [2]:
from lib.readfile import ReadFile as rf
from lib.handle_null_values import handle_null_values as null
from lib.datetime_formatting import DatetimeFormatting as dfmt
from lib.split_time_variable import split_time_variable as split_t
from lib.correct_variable_types import correct_variable_types as vartype

In [3]:
"""Read the file"""

r= rf()
df= r.read(address="lib/data/2009.csv")

In [4]:
"""Remove column with null values"""

n= null()
variables= n.delete_var_with_null_more_than(df)

In [5]:
"""drop unwanted variables for now"""

df2= df.drop(*variables)

In [6]:
"""select important variables""" 

important_variables=['year_built', 'bedroom_count', 'bathroom_count','lot_size', 'area', 'city_name', 'community_name', 'covered_parking', 'exterior', 'energy_source','fireplaces_count', 'floor_covering', 'garage','heating_cooling','latitude','longitude', 'property_type', 'roof', 'sold_date', 'sold_price','unfinished_sq_foot', 'topography', 'year_built', 'zip_code', 'county' ]

    
important_variables= list(set(important_variables))
df3= df2.select(*important_variables)


In [7]:
"""concatenate all files"""

files= ["09"]
for f_no in range(10, 19):
    files.append(f_no)    
    
    df4= r.read(address="lib/data/20"+str(f_no)+".csv")
    print(f_no)
    df4= df4.select(*important_variables)
    print(f_no)
    df3 = df3.union(df4)


10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18


In [8]:
"""filter one county from the dataset"""

df5= df3.filter( df3["county"].isin(["King"]))

In [9]:
"""correct time format"""

f= split_t()

#Assuming time variable is in timestamp type
df6= f.run(df5)


In [28]:
"""Split time variable"""


from pyspark.sql.functions import isnan, when, count, col, from_unixtime
from pyspark.sql.types import DoubleType, TimestampType
from pyspark.sql.functions import *


df6_1= df6.withColumn('sold_date', from_unixtime(unix_timestamp('sold_date', "yyyy-MM-DD")).cast(TimestampType()))
df6_1= df6_1.withColumn('sold_date_year', year('sold_date'))
df6_1= df6_1.withColumn('sold_date_month', month('sold_date'))
df6_1= df6_1.withColumn('sold_date_day', dayofmonth('sold_date'))
df6_1= df6_1.withColumn('sold_date_day', dayofweek('sold_date'))
df6_1= df6_1.drop('sold_date')


In [31]:
"""Correct variable types"""

v= vartype()

df7= v.run(df6_1)


In [40]:
"""make training and testing files and save them."""
df_t0= df7.filter( df7["sold_date_year"]<2015)
df_t= df7.filter(df_t0["sold_date_year"]>=2009)
df_tst= df7.filter( df7["sold_date_year"]>=2015)

In [41]:
df_train2= df_t.toPandas()
df_test2= df_tst.toPandas()

In [42]:
df_train2.to_csv("lib/data/KING'S_COUNTY_train5.0.csv")
df_test2.to_csv("lib/data/KING'S_COUNTY_test5.0.csv")

In [13]:
p=df6.limit(500).toPandas()

numeric_columns= []
columns2= df6.columns # only take string variables

In [14]:
len(list(set(columns2)))

37

In [15]:
p['year_built']
# There is a duplication on the 

0      1959
1      1989
2      1995
3      1984
4      2006
       ... 
495    2006
496    1968
497    1975
498    2005
499    1975
Name: year_built, Length: 500, dtype: object

In [ ]:
for c in columns2:
    counter= 0
    e=[]
    float_values= []
    print(c)
    for i in range(500):
        try:
            v= float(p[c][i])
            counter= counter+1
            float_values.append(v)
        except:
            print(str(p[c][i]))
            e.append(str(p[c][i]))

    if len(e)==0:
        numeric_columns.append(c)  
    elif len(set(float_values))<5: 
        if len(set(e))<=1 and len(set(float_values))!=0:
            numeric_columns.append(c)  
    elif len(float_values)/500*100 > 95:
            numeric_columns.append(c)

In [16]:
var=find_numeric_variables_saved_as_string(df6)

NameError: name 'find_numeric_variables_saved_as_string' is not defined

In [ ]:
"""treat duplications"""
df6.select('year_built').show()

In [43]:
# train the deep learning model and put that on the training

def var_types(df7):
    categorical_variables= []
    numerical_variables=[]
    for dt in df7.dtypes:
        if dt[1]=="string":
            categorical_variables.append(dt[0])
        else:
            numerical_variables.append(dt[0])
    return categorical_variables, numerical_variables
        
categorical_variables, numerical_variables= var_types(df7)

In [44]:
"""treat null values using deep learning"""

# add an ID in the dataset
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W
from pyspark import SparkContext
from pyspark.sql import SparkSession
from math import *


spark_context = SparkContext.getOrCreate()
spark = SparkSession(spark_context)

windowSpec = W.orderBy(df7.columns[0])
df7= df7.withColumn("id", F.row_number().over(windowSpec))
df8= df7.createOrReplaceTempView("customer")

#use this ID to filter the data and get results.
base_query = "SELECT * FROM customer where "
total_rows= df7.count()
each_page= 500
total_pages= ceil(total_rows/each_page)

for p in range(total_pages):
    query= base_query + "id > " + str(each_page*p) + " and id< " + str(each_page*(p+1))
    df_spark= spark.sql(query)

    df_pandas= df_spark.toPandas()
    
    #pass this through all the models in the same order and run it.
    
    
    





#save the model for future use.

In [46]:
# train the deep learning model and put that on the training
categorical_variables= []
numerical_variables=[]
all_variables= []
for dt in df7.dtypes:
    if dt[1]=="string":
        categorical_variables.append(dt[0])
    else:
        numerical_variables.append(dt[0])
        
    all_variables.append(dt[0])
        
        

In [13]:
df_train2= df7.toPandas()

In [78]:
df_train2

,address,unfinished_sq_foot,longitude,covered_parking,topography,sewer,energy_source,school_district,bike_score,sold_price,...,transit_score,latitude,sold_date_year,sold_date_month,sold_date_day,sold_datedayofweek,sold_datehour,sold_datemin,sold_datesec,id
0,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,1
1,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,2
2,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,3
3,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,4
4,"""4803 """"O"""" Ct NE""",0.0,-122.213993,2.0,None,G,Natural Gas,Auburn,0.0,250900.0,...,0.0,47.320628,2011,3,30,4,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209022,9981 Avondale Rd NE,0.0,-122.093338,0.0,None,None,Natural Gas,Lake Washington,0.0,270000.0,...,0.0,47.688652,2010,3,8,2,0,0,0,209023
209023,9981 Avondale Rd NE,0.0,-122.093338,0.0,None,None,Natural Gas,Lake Washington,0.0,270000.0,...,0.0,47.688652,2010,3,8,2,0,0,0,209024
209024,9981 Avondale Rd NE,0.0,-122.093338,0.0,None,None,Natural Gas,Lake Washington,0.0,270000.0,...,0.0,47.688652,2010,3,8,2,0,0,0,209025
209025,9981 Avondale Rd NE,0.0,-122.093338,0.0,None,None,Natural Gas,Lake Washington,0.0,270000.0,...,0.0,47.688652,2010,3,8,2,0,0,0,209026


In [65]:
import datawig
#Initialize a SimpleImputer model
df_train = df7.limit(10000).toPandas()
    


In [14]:
count= 209027
train_df= df_train2[int(count*.25):int(count*.5)]
test_df= df_train2[int(count*.5):]

In [ ]:
var4= 

In [47]:
import datawig
results= {}
models={}

for c in categorical_variables: 
    var= all_variables.copy()
    var.remove(c)
    imputer = datawig.SimpleImputer(
        input_columns=var, # column(s) containing information about the column we want to impute
        output_column=c, # the column we'd like to impute values for
        output_path = 'lib/imputer_models4.0/'+str(c) # stores model data and metrics
        )
    imputer.fit(train_df=df_train2, num_epochs=5)
    models[c]= imputer
    #time to test the model with entire file.  
    
#     results[c] = imputer.predict(df_pandas)
    

    

2019-10-11 16:24:17,945 [INFO]  CategoricalEncoder for column city_name                                found only 91 occurrences of value Newcastle
2019-10-11 16:24:17,950 [INFO]  CategoricalEncoder for column city_name                                found only 56 occurrences of value Lake Forest Park
2019-10-11 16:24:17,955 [INFO]  CategoricalEncoder for column city_name                                found only 47 occurrences of value Carnation
2019-10-11 16:24:17,962 [INFO]  CategoricalEncoder for column city_name                                found only 46 occurrences of value Vashon
2019-10-11 16:24:17,966 [INFO]  CategoricalEncoder for column city_name                                found only 45 occurrences of value Black Diamond
2019-10-11 16:24:17,973 [INFO]  CategoricalEncoder for column city_name                                found only 42 occurrences of value Pacific
2019-10-11 16:24:17,978 [INFO]  CategoricalEncoder for column city_name                                fou

In [ ]:
#impute numerical variables

for c in numerical_variables: 
    var= all_variables.copy()
    var.remove(c)
    imputer = datawig.SimpleImputer(
        input_columns=var, # column(s) containing information about the column we want to impute
        output_column=c, # the column we'd like to impute values for
        output_path = 'lib/imputer_models4.0/'+str(c) # stores model data and metrics
        )
    imputer.fit(train_df=df_train2, num_epochs=5)
    models[c]= imputer
    #time to test the model with entire file.  
    
#     results[c] = imputer.predict(df_pandas)
   

2019-10-11 16:51:14,345 [INFO]  
========== start: fit model
2019-10-11 16:51:14,356 [WARNING]  Already bound, ignoring bind()
2019-10-11 16:51:16,118 [INFO]  Epoch[0] Batch [0-415]	Speed: 3816.39 samples/sec	cross-entropy=7.472362	bedroom_count-accuracy=0.000000
2019-10-11 16:51:17,857 [INFO]  Epoch[0] Train-cross-entropy=5.822307
2019-10-11 16:51:17,859 [INFO]  Epoch[0] Train-bedroom_count-accuracy=0.000000
2019-10-11 16:51:17,861 [INFO]  Epoch[0] Time cost=3.501
2019-10-11 16:51:17,872 [INFO]  Saved checkpoint to "lib/imputer_models4.0/bedroom_count/model-0000.params"
2019-10-11 16:51:18,186 [INFO]  Epoch[0] Validation-cross-entropy=43.506680
2019-10-11 16:51:18,188 [INFO]  Epoch[0] Validation-bedroom_count-accuracy=0.000000
2019-10-11 16:51:19,997 [INFO]  Epoch[1] Batch [0-415]	Speed: 3682.89 samples/sec	cross-entropy=6.470076	bedroom_count-accuracy=0.000000
2019-10-11 16:51:21,925 [INFO]  Epoch[1] Train-cross-entropy=5.480770
2019-10-11 16:51:21,926 [INFO]  Epoch[1] Train-bedroom_

In [74]:
results.to_csv("lib/clean.csv")

In [ ]:
df7.select('address').show

In [ ]:
"""encode categorical variables."""

In [ ]:
"""split data in to training and testing dataset"""

In [ ]:
p=df.limit(500).toPandas()

In [ ]:
p["ml_number"][1]

In [15]:
#Run h2o library offfline

In [19]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd


In [29]:
# d1= pd.read_csv("lib/data/king_train.csv", encoding = 'utf-8')
# d2= pd.read_csv("lib/data/king_test.csv", encoding = 'utf-8')
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_191"; Java(TM) SE Runtime Environment (build 1.8.0_191-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from /Users/muddassarsharif/anaconda3/envs/conda1/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/2h/ylrmltv95sd1_w40dsmy51m00000gn/T/tmpcjhg43s_
  JVM stdout: /var/folders/2h/ylrmltv95sd1_w40dsmy51m00000gn/T/tmpcjhg43s_/h2o_muddassarsharif_started_from_python.out
  JVM stderr: /var/folders/2h/ylrmltv95sd1_w40dsmy51m00000gn/T/tmpcjhg43s_/h2o_muddassarsharif_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.6
H2O cluster version age:,"7 days, 13 hours and 53 minutes"
H2O cluster name:,H2O_from_python_muddassarsharif_l2tzbd
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [113]:
train = h2o.import_file("lib/data/king_train3.0.csv")
test = h2o.import_file("lib/data/king_test3.0.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [114]:
# Identify predictors and response
x = train.columns
y = 'sold_price'
x.remove(y)

In [115]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=10, seed=100)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [116]:
preds = aml.leader.predict(test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


/Users/muddassarsharif/anaconda3/envs/conda1/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'was_flip_imputed' has levels not trained on: [8/2/2012 0:00]
  warnings.warn(w)


In [117]:
p= preds.as_data_frame()

In [130]:
lst_p = list(np.array(p['predict']))
lst_a = list(np.array(test.as_data_frame()['sold_price']))

new_lst = []
mape= []
for i in range(len(lst_p)):
    mape.append(float(abs(lst_p[i]-lst_a[i])/lst_a[i]))
    new_lst.append(float(lst_p[i]))


In [131]:
re= test.as_data_frame()

In [132]:
re["sold_price_predict"]= new_lst
re["APE"]= mape

In [133]:
re.to_csv("lib/data/POC-draft3.1.csv")

In [122]:
model_path = h2o.save_model(model=aml.leader, path="lib/data/mymodel3.0", force=True)

In [ ]:
#read the file.

fn1= pd.read_csv("lib/data/final_data.csv", encoding = 'utf-8')


In [108]:
#view all the collumns
colm= fn1.columns

#make a list of all the categorical columns. 
no_n=[ 'Unnamed: 0', 'sewer', 'address', 'created_date', 'sold_datehour', 'sold_datemin', 'sold_datesec' ]   

#delete all unnecessary columns, including the one that make no senese.
for cv in categorical_variables:
    no_n.append(cv+'_imputed_proba')
    no_n.append(cv)
    
fn2= fn1.drop(no_n, axis=1)
 

In [112]:
#make train and trest file 
tr1= fn2[fn2.sold_date_year< 2013]
tr2= fn2[fn2.sold_date_year>= 2013]

#save the file and run the model.
tr1.to_csv("lib/data/king_train3.0.csv")
tr2.to_csv("lib/data/king_test3.0.csv")

In [126]:
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20191010_114051


Model Summary: 


,,number_of_trees
0,,166.0




ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 5412782969.180901
RMSE: 73571.61795951548
MAE: 47619.186698150166
RMSLE: 0.18206461188304052
Mean Residual Deviance: 5412782969.180901

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 51771826532.961716
RMSE: 227534.231562993
MAE: 84206.09187786179
RMSLE: 0.2618846020084745
Mean Residual Deviance: 51771826532.961716

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,84206.09,3271.1707,86922.92,81333.84,88513.32,82142.664,82117.72
1,mean_residual_deviance,5.1771953E10,2.92360868E10,4.383191E10,3.30569974E10,1.03616528E11,3.9717253E10,3.8637081E10
2,mse,5.1771953E10,2.92360868E10,4.383191E10,3.30569974E10,1.03616528E11,3.9717253E10,3.8637081E10
3,r2,0.6613341,0.055395875,0.68199027,0.67748487,0.56335485,0.6842938,0.69954675
4,residual_deviance,5.1771953E10,2.92360868E10,4.383191E10,3.30569974E10,1.03616528E11,3.9717253E10,3.8637081E10
5,rmse,221785.36,56824.367,209360.72,181815.83,321895.22,199291.88,196563.17
6,rmsle,0.26156583,0.01444027,0.2851953,0.25485155,0.25314474,0.2651821,0.2494555



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2019-10-10 12:12:21,31 min 29.913 sec,0.0,573587.617543,427246.601412,3.290028e+11
1,,2019-10-10 12:12:25,31 min 34.301 sec,5.0,460079.819430,331246.898265,2.116734e+11
2,,2019-10-10 12:12:29,31 min 38.495 sec,10.0,372750.828014,256944.869008,1.389432e+11
3,,2019-10-10 12:12:33,31 min 42.615 sec,15.0,305756.695149,200006.327050,9.348716e+10
4,,2019-10-10 12:12:37,31 min 46.554 sec,20.0,254337.491618,157385.961131,6.468756e+10
5,,2019-10-10 12:12:41,31 min 50.649 sec,25.0,215042.209433,126526.513321,4.624315e+10
6,,2019-10-10 12:12:46,31 min 54.667 sec,30.0,184950.454207,105029.110829,3.420667e+10
7,,2019-10-10 12:12:50,31 min 59.082 sec,35.0,162047.057844,90204.767611,2.625925e+10
8,,2019-10-10 12:12:54,32 min 3.528 sec,40.0,144508.719758,80268.421292,2.088277e+10
9,,2019-10-10 12:12:59,32 min 7.748 sec,45.0,130925.706575,73453.513780,1.714154e+10



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,bathroom_count,4.039431e+16,1.000000,0.218112
1,fireplaces_count,1.443581e+16,0.357372,0.077947
2,latitude,1.425594e+16,0.352920,0.076976
3,area,1.156863e+16,0.286393,0.062466
4,longitude,9.430522e+15,0.233462,0.050921
5,lot_size,9.037170e+15,0.223724,0.048797
6,"floor_covering_imputed.CeramicTile,Fir/Softwood,Hardwood,Vinyl,Wal...",8.232490e+15,0.203803,0.044452
7,year_built,6.966130e+15,0.172453,0.037614
8,"energy_source_imputed.NaturalGas,Solar(Unspecified)",5.299851e+15,0.131203,0.028617
9,bedroom_count,3.728186e+15,0.092295,0.020131



See the whole table with table.as_data_frame()


In [127]:
#read the file.

fn1= pd.read_csv("lib/data/king.csv", encoding = 'utf-8')

1